# Tutorial 5: Neural Architecture Search (NAS) with Mase and Optuna

In this tutorial, we'll see how Mase can be integrated with Optuna, the popular hyperparameter optimization framework, to search for a Bert model optimized for sequence classification on the IMDb dataset. We'll take the Optuna-generated model and import it into Mase, then run the CompressionPipeline to prepare the model for edge deployment by quantizing and pruning its weights.

As we'll see, running Architecture Search with Mase/Optuna involves the following steps.

1. **Define the search space**: this is a dictionary containing the range of values for each parameter at each layer in the model.

2. **Write the model constructor**: this is a function which uses Optuna utilities to sample a model from the search space, and constructs the model using transformers from_config class method.

3. **Write the objective function**: this function calls on the model constructor defined in Step 2 and defines the training/evaluation setup for each search iteration.

4. **Go!** Choose an Optuna sampler, create a study and launch the search.

In [7]:
checkpoint = "prajjwal1/bert-tiny"
tokenizer_checkpoint = "bert-base-uncased"
dataset_name = "imdb"

First, fetch the dataset using the `get_tokenized_dataset` utility.

In [8]:
from chop.tools import get_tokenized_dataset

dataset, tokenizer = get_tokenized_dataset(
    dataset=dataset_name,
    checkpoint=tokenizer_checkpoint,
    return_tokenizer=True,
)

INFO     Tokenizing dataset imdb with AutoTokenizer for bert-base-uncased.


## 1. Defining the Search Space

We'll start by defining a search space, i.e. enumerating the possible combinations of hyperparameters that Optuna can choose during search. We'll explore the following range of values for the model's hidden size, intermediate size, number of layers and number of heads, inspired by the [NAS-BERT paper](https://arxiv.org/abs/2105.14444).

In [9]:
import torch.nn as nn
from chop.nn.modules import Identity

search_space = {
    "num_layers": [2, 4],
    "num_heads": [4, 8],
    "hidden_size": [256, 384, 512],
    "intermediate_size": [512, 768, 1024, 1536, 2048],
}

## 2. Writing a Model Constructor

We define the following function, which will get called in each iteration of the search process. The function is passed the `trial` argument, which is an Optuna object that comes with many functionalities - see the [Trial documentation](https://optuna.readthedocs.io/en/stable/reference/trial.html) for more details. Here, we use the `trial.suggest_int` and `trial.suggest_categorical` functions to trigger the chosen sampler to choose parameter choices and layer types. The suggested integer is the index into the search space for each parameter, which we defined in the previous cell.

In [3]:
# from transformers import AutoConfig, AutoModelForSequenceClassification
# from chop.tools.utils import deepsetattr
# import random

# def construct_model(trial):
#     config = AutoConfig.from_pretrained(checkpoint)

#     # Update the paramaters in the config
#     for param in [
#         "num_layers",
#         "num_heads",
#         "hidden_size",
#         "intermediate_size",
#     ]:
#         # Use suggest_categorical to pick from discrete values (required for GridSampler)
#         chosen_param = trial.suggest_categorical(param, search_space[param])
#         setattr(config, param, chosen_param)

#     trial_model = AutoModelForSequenceClassification.from_config(config)

#     for name, layer in trial_model.named_modules():
#         if isinstance(layer, nn.Linear) and layer.in_features == layer.out_features:
#             new_layer_cls_name = random.choice(["linear", "Identity"])

#             if new_layer_cls_name == "linear":
#                 new_layer_cls = nn.Linear
#             elif new_layer_cls_name == "identity":
#                 new_layer_cls = Identity

#             if new_layer_cls == nn.Linear:
#                 continue
#             elif new_layer_cls == Identity:
#                 new_layer = Identity()
#                 deepsetattr(trial_model, name, new_layer)
#             else:
#                 raise ValueError(f"Unknown layer type: {new_layer_cls}")

#     return trial_model
import torch
from transformers import AutoConfig, AutoModelForSequenceClassification
from chop.tools.utils import deepsetattr

                # new_layer_cls = trial.suggest_categorical(
                #     f"{name}_type",
                #     search_space["linear_layer_choices"],
                # )
layer_choices = ["Linear", "Identity", "Linear", "Linear", "Linear", "Linear", "Linear", "Identity", "Linear"]
def construct_model(trial):
    config = AutoConfig.from_pretrained(checkpoint)

    # Update the paramaters in the config
    for param in [
        "num_layers",
        "num_heads",
        "hidden_size",
        "intermediate_size",
    ]:
        chosen_idx = trial.suggest_categorical(param, search_space[param])
        setattr(config, param, chosen_idx)

    trial_model = AutoModelForSequenceClassification.from_config(config)
    j = 0
    for name, layer in trial_model.named_modules():
        if isinstance(layer, nn.Linear) and layer.in_features == layer.out_features:
            new_layer_cls_name = layer_choices[j % len(layer_choices)]
            j += 1
            print(j)
            if new_layer_cls_name == "Linear":
                continue
            elif new_layer_cls_name == "Identity":
                new_layer = Identity()
                deepsetattr(trial_model, name, new_layer)
            else:
                raise ValueError(f"Unknown layer type: {new_layer_cls_name}")

    device = "cuda" if torch.cuda.is_available() else "cpu"
    return trial_model.to(device)


## 3. Defining the Objective Function

Next, we define the objective function for the search, which gets called on each trial. In each trial, we create a new model instace with chosen hyperparameters according to the defined sampler. We then use the `get_trainer` utility in Mase to run a training loop on the IMDb dataset for a number of epochs. Finally, we use `evaluate` to report back the classification accuracy on the test split.

In [10]:
from chop.tools import get_trainer


def objective(trial):

    # Define the model
    model = construct_model(trial)

    trainer = get_trainer(
        model=model,
        tokenized_dataset=dataset,
        tokenizer=tokenizer,
        evaluate_metric="accuracy",
        num_train_epochs=1,
    )

    trainer.train()
    eval_results = trainer.evaluate()

    # Set the model as an attribute so we can fetch it later
    trial.set_user_attr("model", model)

    return eval_results["eval_accuracy"]

## 4. Launching the Search

Optuna provides a number of samplers, for example:

* **GridSampler**: iterates through every possible combination of hyperparameters in the search space
* **RandomSampler**: chooses a random combination of hyperparameters in each iteration
* **TPESampler**: uses Tree-structured Parzen Estimator algorithm to choose hyperparameter values.

You can define the chosen sampler by simply importing from `optuna.samplers` as below.

In [11]:
from optuna.samplers import GridSampler, RandomSampler, TPESampler

sampler = GridSampler(search_space)

With all the pieces in place, we can launch the search as follows. The number of trials is set to 1 so you can go get a coffee for 10 minutes, then proceed with the tutorial. However, this will essentially be a random model - for better results, set this to 100 and leave it running overnight!

In [12]:
import optuna
from pathlib import Path
import dill

# Create a single study
study = optuna.create_study(
    direction="maximize",
    study_name="bert-tiny-nas-study",
    sampler=sampler,
)

# Callback function to save best model after each trial
def save_best_model_callback(study, trial):
    trial_num = len(study.trials)
    print(f"Completed trial {trial_num}/30")
    
    if study.best_trial:
        model = study.best_trial.user_attrs["model"].cpu()
        with open(f"{Path.home()}/tutorial_5_best_model_{trial_num}_GridSampler.pkl", "wb") as f:
            dill.dump(model, f)
        print(f"Best accuracy so far: {study.best_value:.4f}")

# Run 30 trials with callback
study.optimize(
    objective,
    n_trials=30,
    timeout=60 * 60 * 24,
    callbacks=[save_best_model_callback]
)

# Print all study values at the end
print("\nAll trial results:")
for t in study.trials:
    print(f"Trial {t.number}: Value = {t.value:.4f}, Params = {t.params}")



[I 2026-02-06 00:22:10,460] A new study created in memory with name: bert-tiny-nas-study


1
2
3
4
5
6
7
8
9


/home/dt822/adls/src/chop/tools/huggingface.py:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/dt822/adls/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
500,0.593800


/home/dt822/adls/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/dt822/adls/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


...

/home/dt822/adls/src/chop/tools/huggingface.py:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/dt822/adls/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
500,0.583500


/home/dt822/adls/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/dt822/adls/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


[I 2026-02-06 00:52:43,818] Trial 29 finished with value: 0.8146 and parameters: {'num_layers': 4, 'num_heads': 8, 'hidden_size': 384, 'intermediate_size': 1024}. Best is trial 16 with value: 0.83768.


Completed trial 30/30
Best accuracy so far: 0.8377

All trial results:
Trial 0: Value = 0.8098, Params = {'num_layers': 2, 'num_heads': 8, 'hidden_size': 384, 'intermediate_size': 768}
Trial 1: Value = 0.8094, Params = {'num_layers': 4, 'num_heads': 8, 'hidden_size': 384, 'intermediate_size': 1536}
Trial 2: Value = 0.8260, Params = {'num_layers': 4, 'num_heads': 8, 'hidden_size': 512, 'intermediate_size': 2048}
Trial 3: Value = 0.8146, Params = {'num_layers': 2, 'num_heads': 4, 'hidden_size': 384, 'intermediate_size': 1024}
Trial 4: Value = 0.8083, Params = {'num_layers': 4, 'num_heads': 8, 'hidden_size': 256, 'intermediate_size': 1024}
Trial 5: Value = 0.7932, Params = {'num_layers': 2, 'num_heads': 8, 'hidden_size': 256, 'intermediate_size': 512}
Trial 6: Value = 0.8094, Params = {'num_layers': 2, 'num_heads': 8, 'hidden_size': 384, 'intermediate_size': 1536}
Trial 7: Value = 0.8260, Params = {'num_layers': 2, 'num_heads': 8, 'hidden_size': 512, 'intermediate_size': 2048}
Trial 8: Va

Fetch the model associated with the best trial as follows, and export to be used in future tutorials. In Tutorial 6, we'll see how to run mixed-precision quantization search on top of the model we've just found through NAS to further find the optimal quantization mapping.

In [ ]:
from pathlib import Path
import dill

model = study.best_trial.user_attrs["model"].cpu()

with open(f"{Path.home()}/tutorial_5_best_model.pkl", "wb") as f:
    dill.dump(model, f)

In [ ]:
# Print all study values at the end
print("\nAll trial results:")
for t in study.trials:
    print(f"Trial {t.number}: Value = {t.value:.4f}, Params = {t.params}")

## Deploying the Optimized Model with CompressionPipeline

Now, we can run the CompressionPipeline in Mase to run uniform quantization and pruning over the searched model.

In [7]:
from chop.pipelines import CompressionPipeline
from chop import MaseGraph
import dill
from pathlib import Path

# Load the best model
with open(f"{Path.home()}/tutorial_5_best_model_30_TPESampler.pkl", "rb") as f:
    model = dill.load(f)


mg = MaseGraph(model)
pipe = CompressionPipeline()

quantization_config = {
    "by": "type",
    "default": {
        "config": {
            "name": None,
        }
    },
    "linear": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
    },
}

pruning_config = {
    "weight": {
        "sparsity": 0.5,
        "method": "l1-norm",
        "scope": "local",
    },
    "activation": {
        "sparsity": 0.5,
        "method": "l1-norm",
        "scope": "local",
    },
}

# mg, _ = pipe(
#     mg,
#     pass_args={
#         "quantize_transform_pass": quantization_config,
#         "prune_transform_pass": pruning_config,
#     },
# )

`past_key_values` were not specified as input names, but model.config.use_cache = True. Setting model.config.use_cache = False.


In [ ]:
from chop.tools import get_trainer
import torch
import copy

def objective(trial):

    # Define the model
    model = construct_model(trial)

    trainer = get_trainer(
        model=model,
        tokenized_dataset=dataset,
        tokenizer=tokenizer,
        evaluate_metric="accuracy",
        num_train_epochs=1,
    )

    trainer.train()

    # Ensure model is on CPU before creating MaseGraph
    model = model.to("cpu")
    
    mg = MaseGraph(model, hf_input_names=[
        "input_ids",
        "attention_mask",
        "labels",
    ])
    
    # Deep copy configs since pipe() modifies them in-place
    compressed_mg, _ = pipe(mg, pass_args={
        "quantize_transform_pass": copy.deepcopy(quantization_config),
        "prune_transform_pass": copy.deepcopy(pruning_config),
    })

    compressed_trainer = get_trainer(
        model=compressed_mg.model,
        tokenized_dataset=dataset,
        tokenizer=tokenizer,
        evaluate_metric="accuracy",
        num_train_epochs=1,
    )

    compressed_trainer.train()

    eval_results = compressed_trainer.evaluate()

    # Store the underlying model (not MaseGraph) for serialization compatibility
    trial.set_user_attr("model", compressed_mg.model.state_dict())

    return eval_results["eval_accuracy"]



# Train your baseline for 2 epochs
# Quantize


# See if we can recover our loss compared to our baseline

In [9]:
from optuna.samplers import GridSampler, RandomSampler, TPESampler

sampler = TPESampler()

In [10]:
import torch
import optuna
from pathlib import Path
import dill

# Create a single study
study = optuna.create_study(
    direction="maximize",
    study_name="bert-tiny-nas-study",
    sampler=sampler,
)

# Callback function to save best model after each trial
def save_best_model_callback(study, trial):
    trial_num = len(study.trials)
    print(f"Completed trial {trial_num}/30")
    
    if study.best_trial:
        model = study.best_trial.user_attrs["model"]
        torch.save(model, f"{Path.home()}/tutorial_5_best_model_{trial_num}_TPESampler_quantised_and_trained.pt")
        print(f"Best accuracy so far: {study.best_value:.4f}")

# Run 30 trials with callback
study.optimize(
    objective,
    n_trials=30,
    timeout=60 * 60 * 24,
    callbacks=[save_best_model_callback]
)

# Print all study values at the end
print("\nAll trial results:")
for t in study.trials:
    print(f"Trial {t.number}: Value = {t.value:.4f}, Params = {t.params}")

[I 2026-02-03 20:08:05,486] A new study created in memory with name: bert-tiny-nas-study
/home/dt822/adls/.venv/lib/python3.11/site-packages/optuna/distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.nn.modules.linear.Linear'> which is of type type.
  warnings.warn(message)
/home/dt822/adls/.venv/lib/python3.11/site-packages/optuna/distributions.py:502: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'chop.nn.modules.identity.Identity'> which is of type type.
  warnings.warn(message)
/home/dt822/adls/src/chop/tools/huggingface.py:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/dt822/adls/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.

Step,Training Loss
500,0.693300


/home/dt822/adls/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
`past_key_values` were not specified as input names, but model.config.use_cache = True. Setting model.config.use_cache = False.
INFO     Getting dummy input for prajjwal1/bert-tiny.
INFO     Pruning module: bert_encoder_layer_0_attention_self_key
INFO     Pruning module: bert_encoder_layer_0_attention_self_value
INFO     Pruning module: bert_encoder_layer_0_attention_output_dense
INFO     Pruning module: bert_encoder_layer_0_intermediate_dense
INFO     Pruning module: bert_encoder_layer_0_output_dense
INFO     Pruning module: bert_encoder_layer_1_attention_self_value
INFO     Pruning module: bert_encoder_layer_1_intermediate_dense
INFO     Pruning module: bert_encoder_layer_1_output_dense
INFO     Pruning module: bert_pooler_dense
INFO     Pruning modul

tensor([[ 101, 9932, 2089, 2202, 2058, 1996, 2088, 2028, 2154,  102],
        [ 101, 2023, 2003, 2339, 2017, 2323, 4553, 4748, 4877,  102]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[ 101, 9932, 2089, 2202, 2058, 1996, 2088, 2028, 2154,  102],
        [ 101, 2023, 2003, 2339, 2017, 2323, 4553, 4748, 4877,  102]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]],


        [[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]]])
tensor([[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       

/home/dt822/adls/src/chop/tools/huggingface.py:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/dt822/adls/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


...

/home/dt822/adls/src/chop/tools/huggingface.py:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/dt822/adls/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
500,0.478900


/home/dt822/adls/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/dt822/adls/.venv/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


[I 2026-02-03 21:20:42,640] Trial 29 finished with value: 0.80932 and parameters: {'num_layers': 1, 'num_heads': 0, 'hidden_size': 1, 'intermediate_size': 0, 'bert.encoder.layer.0.attention.self.query_type': <class 'chop.nn.modules.identity.Identity'>, 'bert.encoder.layer.0.attention.self.key_type': <class 'torch.nn.modules.linear.Linear'>, 'bert.encoder.layer.0.attention.self.value_type': <class 'torch.nn.modules.linear.Linear'>, 'bert.encoder.layer.0.attention.output.dense_type': <class 'torch.nn.modules.linear.Linear'>, 'bert.encoder.layer.1.attention.self.query_type': <class 'chop.nn.modules.identity.Identity'>, 'bert.encoder.layer.1.attention.self.key_type': <class 'chop.nn.modules.identity.Identity'>, 'bert.encoder.layer.1.attention.self.value_type': <class 'torch.nn.modules.linear.Linear'>, 'bert.encoder.layer.1.attention.output.dense_type': <class 'chop.nn.modules.identity.Identity'>, 'bert.pooler.dense_type': <class 'chop.nn.modules.identity.Identity'>}. Best is trial 20 with 

Completed trial 30/30
Best accuracy so far: 0.8698

All trial results:
Trial 0: Value = 0.7561, Params = {'num_layers': 1, 'num_heads': 0, 'hidden_size': 0, 'intermediate_size': 0, 'bert.encoder.layer.0.attention.self.query_type': <class 'chop.nn.modules.identity.Identity'>, 'bert.encoder.layer.0.attention.self.key_type': <class 'torch.nn.modules.linear.Linear'>, 'bert.encoder.layer.0.attention.self.value_type': <class 'torch.nn.modules.linear.Linear'>, 'bert.encoder.layer.0.attention.output.dense_type': <class 'torch.nn.modules.linear.Linear'>, 'bert.encoder.layer.1.attention.self.query_type': <class 'chop.nn.modules.identity.Identity'>, 'bert.encoder.layer.1.attention.self.key_type': <class 'chop.nn.modules.identity.Identity'>, 'bert.encoder.layer.1.attention.self.value_type': <class 'torch.nn.modules.linear.Linear'>, 'bert.encoder.layer.1.attention.output.dense_type': <class 'chop.nn.modules.identity.Identity'>, 'bert.pooler.dense_type': <class 'torch.nn.modules.linear.Linear'>}
Tri

In [ ]:
mg.export(f"{Path.home()}/tutorial_5_nas_compressed", save_format="state_dict")